## Okay, dann starten wir mit dem replizieren der Ergebnisse!
Als erstes wüssten wir gerne, ob die Werte der top 1-Accuracy und der Variational Distance mit denen auf Tabelle 13,17,14,18
Fangen wir an die Werte der Variational Distance zu verlgeichen mit denen aus Tabelle 14 und 18

In [1]:
import torch
import pandas as pd
import numpy as np
import sentencepiece as spm
import gcsfs

# Sicherstellen, dass eine GPU verfügbar ist
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Verbindung zu Google Cloud Storage
fs = gcsfs.GCSFileSystem('transformer-ngrams')

# Tokenizer laden
tokenizer_path = 'gs://transformer-ngrams/32768.model'
with fs.open(tokenizer_path) as f:
    tokenizer = spm.SentencePieceProcessor(model_proto=f.read())

Using device: cuda


In [2]:
# Hilfsfunktion zur Konvertierung von Token-Statistiken in Wahrscheinlichkeiten
def convert_counter_to_probs(counter):
    """Konvertiert Token-Zähler in Wahrscheinlichkeiten."""
    assert len(counter) % 2 == 0
    probs = np.zeros(32768)
    ks = counter[::2]  # Token IDs
    vs = counter[1::2]  # Vorkommen
    mass = sum(vs)
    for k, v in zip(ks, vs):
        probs[k] = v / mass
    return probs

# Hilfsfunktion zur Berechnung der Variational Distance
def dist(counter, model_probs):
    """Berechnet die Variational Distance zwischen Regel-basierten Vorhersagen und Modellvorhersagen."""
    probs = convert_counter_to_probs(counter)
    return 0.5 * np.sum(np.abs(probs - model_probs))

Um zu sehen ob die Hilfsfunktionen funktionieren, ist hier ein kleiner test, der gut aussieht

In [8]:
# Testdaten: Ein Beispiel-Token-Zähler [Token1, Count1, Token2, Count2, ...]
test_counter = [100, 5, 200, 10, 300, 15]  
test_model_probs = np.random.uniform(0, 1, 32768)  
test_model_probs /= test_model_probs.sum()  # Normierung auf eine Wahrscheinlichkeitsverteilung  

# Test für convert_counter_to_probs
probs = convert_counter_to_probs(test_counter)
print("Token-Probs aus Zähler:", probs[100], probs[200], probs[300])  # Erwartete Werte: Nicht null

# Test für dist
distance = dist(test_counter, test_model_probs)
print("Berechnete Variational Distance:", distance)  # Erwartet: Float-Wert

Token-Probs aus Zähler: 0.16666666666666666 0.3333333333333333 0.5
Berechnete Variational Distance: 0.9998585525356616


In [3]:
# Schritt 1: Laden der Regeln-Daten für Tabelle 13
print("Laden der Regeln-Daten für Tabelle 13...")
sample_rules_path = 'gs://transformer-ngrams/TinyStories/eval_data_rules/001.parquet'
with fs.open(sample_rules_path, 'rb') as f:
    df_rules = pd.read_parquet(f)
print("Regeln-Daten geladen:")
print(df_rules.head())

# Schritt 2: Dummy-Modellvorhersagen erstellen (wird später durch ein echtes Modell ersetzt)
n = len(set(df_rules['index']))
index = list(range(n))
random_probs = np.random.uniform(size=(n, 32768))
random_probs = random_probs / np.sum(random_probs, axis=1)[:, None]
df_model_preds = pd.DataFrame({'index': index, 'model_probs': random_probs.tolist()})

Laden der Regeln-Daten für Tabelle 13...
Regeln-Daten geladen:
   record_num  index  token  target  context_size_used     context rules  \
0           0      0      1   31402                  1         [1]   [1]   
3           0      1  31402   32599                  1     [31402]   [1]   
4           0      1  31402   32599                  2         [1]   [2]   
2           0      1  31402   32599                  2      [1, 0]   [3]   
1           0      1  31402   32599                  2  [1, 31402]   [4]   

                                  next_token_counter rule_prediction  text  \
0  [32606, 38614, 32624, 132017, 32318, 40357, 41...          [4146]         
3  [1358, 56, 2508, 453, 305, 935, 11655, 2, 3260...         [32600]  Spot   
4  [32606, 38614, 32624, 132017, 32318, 40357, 41...          [4146]  Spot   
2  [2904, 36075, 7097, 16560, 305, 151603, 1726, ...          [1726]  Spot   
1                [383, 40, 8130, 1, 328, 9, 5130, 1]           [383]  Spot   

  target_te

In [9]:
print(df_model_preds.shape)  # Erwartet: (Anzahl der Token, 2)
print(df_model_preds.head()) # Vorschau der zufälligen Modellvorhersagen


(85, 2)
   index                                        model_probs
0      0  [7.501295998503143e-06, 2.016407590672307e-06,...
1      1  [3.429055587820258e-05, 2.3110939107225145e-05...
2      2  [3.057854857864653e-05, 4.9317750988683684e-05...
3      3  [6.049454313797489e-05, 3.1473602833666823e-05...
4      4  [4.0739463675605206e-05, 4.079556770278851e-05...


In [4]:
# Schritt 3: Berechnung der Variational Distance
print("Berechnung der Variational Distance für Tabelle 13...")
df_joined = df_rules.merge(df_model_preds, on='index')
df_joined['distance'] = df_joined.apply(lambda x: dist(x.next_token_counter, x.model_probs), axis=1)

# Schritt 4: Optimale Regel pro Token auswählen
df_optimal = df_joined.loc[df_joined.groupby('index')['distance'].idxmin()].reset_index(drop=True)
df_optimal['top_1_acc'] = df_optimal.apply(lambda x: (np.argmax(x['model_probs']) in x['rule_prediction']) / len(x['rule_prediction']), axis=1)

Berechnung der Variational Distance für Tabelle 13...


Überprüfen der Werte:

In [10]:
print(df_joined[["index", "distance"]].head())  # Erwartet: Float-Werte
print(df_joined["distance"].describe())  # Erwartet: Min, Max, Mittelwert zur Überprüfung


   index  distance
0      0  0.993689
1      1  0.987740
2      1  0.993422
3      1  0.985944
4      1  0.999896
count    28073.000000
mean         0.987931
std          0.021106
min          0.898294
25%          0.986984
50%          0.998433
75%          0.999808
max          1.000000
Name: distance, dtype: float64


In [5]:
# Ergebnis für Tabelle 13 anzeigen
print("Ergebnis für Tabelle 13:")
print(df_optimal.head())


Ergebnis für Tabelle 13:
   record_num  index  token  target  context_size_used        context rules  \
0           0      0      1   31402                  1            [1]   [1]   
1           0      1  31402   32599                  2         [1, 0]   [3]   
2           0      2  32599   15993                  2     [31402, 0]   [3]   
3           0      3  15993    2531                  2     [32599, 0]   [3]   
4           0      4   2531     280                  3  [32599, 0, 0]   [9]   

                                  next_token_counter rule_prediction   text  \
0  [32606, 38614, 32624, 132017, 32318, 40357, 41...          [4146]          
1  [2904, 36075, 7097, 16560, 305, 151603, 1726, ...          [1726]   Spot   
2  [496, 596, 306, 1771, 18449, 136, 280, 1064, 3...           [306]      .   
3  [382, 474977, 12779, 4839, 597, 79146, 4, 6577...             [4]   Spot   
4  [841, 37385, 4618, 635, 25700, 24, 32616, 1258...           [306]    saw   

  target_text            

In [13]:
for _, row in df_optimal.iterrows():
    print(f"Index: {row['index']}, Rule Prediction: {row['rule_prediction']}, Model Prediction: {np.argmax(row['model_probs'])}")



Index: 0, Rule Prediction: [4146], Model Prediction: 7241
Index: 1, Rule Prediction: [1726], Model Prediction: 5700
Index: 2, Rule Prediction: [306], Model Prediction: 29698
Index: 3, Rule Prediction: [4], Model Prediction: 14321
Index: 4, Rule Prediction: [306], Model Prediction: 29697
Index: 5, Rule Prediction: [305], Model Prediction: 20682
Index: 6, Rule Prediction: [305], Model Prediction: 5465
Index: 7, Rule Prediction: [305], Model Prediction: 21673
Index: 8, Rule Prediction: [32599], Model Prediction: 10760
Index: 9, Rule Prediction: [32599], Model Prediction: 18509
Index: 10, Rule Prediction: [32599], Model Prediction: 25024
Index: 11, Rule Prediction: [32599], Model Prediction: 4063
Index: 12, Rule Prediction: [32599], Model Prediction: 11955
Index: 13, Rule Prediction: [32599], Model Prediction: 27693
Index: 14, Rule Prediction: [32599], Model Prediction: 29494
Index: 15, Rule Prediction: [32599], Model Prediction: 17308
Index: 16, Rule Prediction: [32599], Model Prediction:

In [11]:
print(df_optimal[["index", "distance", "top_1_acc"]].head())
print(df_optimal["top_1_acc"].describe())  # Erwartet: Werte zwischen 0 und 1


   index  distance  top_1_acc
0      0  0.993689        0.0
1      1  0.985944        0.0
2      2  0.979184        0.0
3      3  0.939211        0.0
4      4  0.931628        0.0
count    85.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: top_1_acc, dtype: float64


In [6]:
# Schritt 5: Speichern der Ergebnisse für Tabelle 13
df_optimal.to_csv("table_13_results.csv", index=False)

In [7]:
# Falls alles funktioniert, wiederholen wir für Tabelle 14 mit neuen Regeln
print("Laden der Regeln-Daten für Tabelle 14...")
sample_rules_path_14 = 'gs://transformer-ngrams/TinyStories/eval_data_rules/002.parquet'
with fs.open(sample_rules_path_14, 'rb') as f:
    df_rules_14 = pd.read_parquet(f)
print("Regeln-Daten für Tabelle 14 geladen:")
print(df_rules_14.head())


Laden der Regeln-Daten für Tabelle 14...
Regeln-Daten für Tabelle 14 geladen:
   record_num  index  token  target  context_size_used    context rules  \
0           1      0      1    4146                  1        [1]   [1]   
3           1      1   4146    1726                  1     [4146]   [1]   
4           1      1   4146    1726                  2        [1]   [2]   
2           1      1   4146    1726                  2     [1, 0]   [3]   
1           1      1   4146    1726                  2  [1, 4146]   [4]   

                                  next_token_counter rule_prediction  text  \
0  [32606, 38614, 32624, 132017, 32318, 40357, 41...          [4146]         
3  [1726, 1260471, 611, 156123, 280, 2612, 32600,...          [1726]  Once   
4  [32606, 38614, 32624, 132017, 32318, 40357, 41...          [4146]  Once   
2  [2904, 36075, 7097, 16560, 305, 151603, 1726, ...          [1726]  Once   
1  [1726, 1259081, 611, 155779, 32600, 11109, 275...          [1726]  Once   

  